In [1]:
!pip install -U --pre tensorflow=="2.*"
!pip install pycocotools


import tensorflow as tf;
import os
import pathlib

import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

print(tf.__version__);


import tensorflow as tf
from google.colab import drive
drive.mount('/content/gdrive')
print ('mounted on')
!pwd
!ls

print("Using Tensorflow version")
print(tf.__version__)

basedir='/content/gdrive/My Drive/dataset/'


import os
import pathlib

%cd /content/gdrive/My Drive/dataset/models/research
print('Change directory')
!pwd
!ls
!protoc object_detection/protos/*.proto --python_out=.

%cd /content/gdrive/My Drive/dataset/models/research
!pip install .


import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display


from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


print("Patching")

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1
print("Patching completed")
# Patch the location of gfile
tf.gfile = tf.io.gfile
print("g file loaded")



def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model




     |████████████████████████████████| 86.3MB 39kB/s 
     |████████████████████████████████| 3.8MB 38.3MB/s 
     |████████████████████████████████| 450kB 45.2MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalle

2.0.0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
mounted on
/content
gdrive	sample_data
Using Tensorflow version
2.0.0
/content/gdrive/My Drive/dataset/models/research
Change directory
/content/gdrive/My Drive/dataset/models/research
a3c_blogpost			  lm_commonsense
adversarial_crypto		  lstm_object_detection
adversarial_logit_pairing	  marco
adversarial_text		  maskgan
adv_imagenet_models		  morph_net
astronet			  namignizer
attention_ocr			  neural_gpu
audioset			  neural_programmer
autoaugment			  n

In [18]:
import imutils

#PATH_TO_LABELS = '/content/gdrive/My Drive/dataset/models/research/object_detection/data/mscoco_label_map.pbtxt'
PATH_TO_LABELS = '/content/gdrive/My Drive/dataset/models/research/object_detection/data/object_detection.pbtxt'

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


basedir='/content/gdrive/My Drive/dataset/har/'
VIDEO_DIR=basedir+'test-data/'
VIDEO_OUT_DIR=basedir+'output-data'
VIDEO_IN_DIR=basedir+'output-data/videos/'
VIDEO_SAVE_DIR=basedir+'test-data/frames-od/'

PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/gdrive/My Drive/dataset/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))



model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
print(model_name)
detection_model = load_model(model_name)
print("model loaded")

print(detection_model.inputs)

detection_model.output_dtypes
detection_model.output_shapes



def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict



  
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))


frame_count=0
batch_size=1
#frames=[]
capture = cv2.VideoCapture(os.path.join(VIDEO_IN_DIR, 'out.mp4'))    
while True:
    ret, frame = capture.read()
    if not ret:
        break        
    frame = imutils.resize(frame,width=400)
    # Bail out when the video file ends
    orig_frame = frame
    image_np=frame
    # Save each frame of the video to a list
    frame_count += 1

    output_dict = run_inference_for_single_image(detection_model, image_np)
  # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=8)
    display(Image.fromarray(image_np))
    
    img_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    #cv2_imshow(img_rgb)
    name = VIDEO_SAVE_DIR+'{0}.jpg'.format(frame_count)
    print(name)
    cv2.imwrite(name, img_rgb)
 


Output hidden; open in https://colab.research.google.com to view.

In [19]:
import glob

# Get all image file paths to a list.
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by name index.
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

video = cv2.VideoCapture(os.path.join(VIDEO_DIR, '20191116_130301.mp4'));

# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver)  < 3 :
    fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else :
    fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

video.release();




def generate_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

import glob
import os

# Directory of images to run detection on
VIDEO_DIR = os.path.join(VIDEO_OUT_DIR, "videos")
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = os.path.join(VIDEO_DIR, "out3.mp4")
generate_video(outvid, images, fps=30)

Frames per second using video.get(cv2.CAP_PROP_FPS) : 29.85779592112144


<VideoWriter 0x7f7575a86250>